In [1]:
!pip install datasets
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install optimum
!pip install sentencepiece
!pip install torch
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install einops
!pip install ctranslate2


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new 

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!ct2-transformers-converter --model meta-llama/Llama-2-13b-chat-hf --output_dir llama-13b-ct2 --force

Loading checkpoint shards: 100%|██████████████████| 3/3 [00:17<00:00,  5.67s/it]


In [4]:
import transformers
import ctranslate2
tokenizer = transformers.AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf")
generator = ctranslate2.Generator("llama-13b-ct2", device="cuda", compute_type="float16", device_index=[0])

In [6]:
import zipfile
import os

zip_file_path = 'data.zip'
extracted_dir = os.getcwd()
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [213]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [214]:
loader = DirectoryLoader(os.getcwd() + "/data/", glob="*/*.txt", loader_cls=TextLoader)
documents = loader.load()

In [215]:
len(documents)

11

In [216]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                          chunk_overlap=0)
texts = splitter.split_documents(documents)

In [217]:
texts[:5]

[Document(page_content='About the adrenal glands\nThe adrenals are small glands that sit above each of the kidneys. The kidneys are located deep inside the upper part of the abdomen.\nEach adrenal gland has 2 parts. The outer part, the cortex, is where most tumors develop. The cortex makes certain hormones for the body. These hormones all have a similar chemical structure and are called steroids:\n* Cortisol causes changes in metabolism to help the body to handle stress.\n* Aldosterone helps the kidneys regulate the amount of salt in the blood and helps regulate blood pressure.\n* Adrenal androgens can be converted to more common forms of the sex hormones estrogen and testosterone in other parts of the body. The amount of these hormones that result from conversion of adrenal androgens is small compared to what is made in other parts of the body. The testicles produce most of the androgens (male hormones) in men. The ovaries produce most of the estrogens (female hormones) in women.', me

In [269]:
!pip install InstructorEmbedding

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [316]:
import chromadb
import os

In [317]:
persist_directory = 'doccuments_db'
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [318]:
client = chromadb.PersistentClient(path=os.path.join(os.getcwd(), persist_directory))

In [319]:
collection = client.get_or_create_collection(persist_directory, metadata={"hnsw:space": "cosine"})

In [320]:
for idx, text in enumerate(texts):
    
    curent_text = text.page_content
    # print(curent_text)
    embedding = embeddings.embed_query(curent_text)
    collection.add(
        embeddings=[embedding],
        documents=[curent_text], 
        metadatas=[{'metadata': text.metadata['source']}],
        ids=[str(idx)],
    )
    print(f'{idx+1} / {len(texts)}')

1 / 147
2 / 147
3 / 147
4 / 147
5 / 147
6 / 147
7 / 147
8 / 147
9 / 147
10 / 147
11 / 147
12 / 147
13 / 147
14 / 147
15 / 147
16 / 147
17 / 147
18 / 147
19 / 147
20 / 147
21 / 147
22 / 147
23 / 147
24 / 147
25 / 147
26 / 147
27 / 147
28 / 147
29 / 147
30 / 147
31 / 147
32 / 147
33 / 147
34 / 147
35 / 147
36 / 147
37 / 147
38 / 147
39 / 147
40 / 147
41 / 147
42 / 147
43 / 147
44 / 147
45 / 147
46 / 147
47 / 147
48 / 147
49 / 147
50 / 147
51 / 147
52 / 147
53 / 147
54 / 147
55 / 147
56 / 147
57 / 147
58 / 147
59 / 147
60 / 147
61 / 147
62 / 147
63 / 147
64 / 147
65 / 147
66 / 147
67 / 147
68 / 147
69 / 147
70 / 147
71 / 147
72 / 147
73 / 147
74 / 147
75 / 147
76 / 147
77 / 147
78 / 147
79 / 147
80 / 147
81 / 147
82 / 147
83 / 147
84 / 147
85 / 147
86 / 147
87 / 147
88 / 147
89 / 147
90 / 147
91 / 147
92 / 147
93 / 147
94 / 147
95 / 147
96 / 147
97 / 147
98 / 147
99 / 147
100 / 147
101 / 147
102 / 147
103 / 147
104 / 147
105 / 147
106 / 147
107 / 147
108 / 147
109 / 147
110 / 147
111 / 14

In [453]:
import re
import ast

In [371]:
def filter_documents(user_question):
    query_embedding = embeddings.embed_query(user_question)

    query_result = collection.query(
            query_embeddings=[query_embedding],
            n_results=5,
        )
    
    docs = []

    for distance, document, metadata in zip(query_result['distances'][0], query_result['documents'][0], query_result['metadatas'][0]):
        if distance > 0.30:
            continue
    
        docs.append(document)

    return docs
    

In [473]:
def format_as_json(string):
    string = string.replace("'answer'", '"answer"').replace("'}", '"}')
    string = re.sub("\":.*\'", '\": \"', string)
    return string

In [570]:
async def get_initial_answer(docs, user_question):
    context = ""
    for doc in docs[:1]:
        context+=f'{doc}\n'
    
    prompt = f"""
[INST] <<SYS>>
You are an expert AI assistant that answers the questions, based on the provided context. You don't know anything about medicine, so it's important that you give an answer only using the context. If the question is not related to context or there is no answer to that question - say that you can't answer it, because it's out of context.
Answer must be strictly in the below format:

```
{{"answer": "your answer here"}}
```
<</SYS>>
context={context}
question={user_question}
[/INST]
"""
    
    initial = ""
    
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(prompt))
    
    result_genrator = generator.async_generate_tokens(
        [tokens], 
        sampling_topk=10,
        sampling_temperature=0.001,
        max_length=2048, 
    )
    
    async for item in result_genrator:
        initial += item.token + ""
    
    initial = initial.strip().replace('▁', ' ').replace('<0x0A>', '\\n').replace('</s>', '')
    initial = re.findall(r'{.*}', initial)
    if len(initial) > 0:
        initial = initial[0]

    initial = format_as_json(initial)
    initial = json.loads(initial, strict=False)
    initial = initial["answer"]

    return initial

In [589]:
async def refine(docs, intermediate):
    for doc in docs[1:]:
        context = ""
        context+=f'{doc}\n'
        
        prompt = f"""
[INST] <<SYS>>
Look at the intermediate answer below. Given the new context add new details to the intermediate answer. You must add details, that are present in the context, do not use your own knowledge.
Answer must be strictly in the below format:

```
{{"answer": "your answer here"}}
```
<</SYS>>
context: {context}
intermediate answer: {intermediate}
[/INST]
"""
        res = ""
        tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(prompt))
        result_genrator = generator.async_generate_tokens(
            [tokens], 
            sampling_topk=10,
            sampling_temperature=0.001,
            max_length=4096, 
        )
        async for item in result_genrator:
            res += item.token + ""
        intermediate = res.strip().replace('▁', ' ').replace('<0x0A>', '\\n').replace('</s>', '')        
        intermediate = re.findall(r'{.*}', intermediate)
        
        if len(intermediate) > 0:
            intermediate = intermediate[0]
        try:
            intermediate = json.loads(intermediate, strict=False)
        except:
            format_as_json(intermediate)

        # print(intermediate)
        
        intermediate = intermediate["answer"]

    return intermediate

In [590]:
async def answer_question(user_question):

    docs = filter_documents(user_question)
    
    if len(docs) == 0:
        return f"Sorry, I don't know the answer to \"{user_question}\""
        
    initial = await get_initial_answer(docs, user_question)
    final = await refine(docs, initial)
    
    
        
    return final

Question, present in the knowledge base

In [591]:
user_question = "What are the symptoms of bladder cancer?"

import time
import re

start = time.time()
answer = await answer_question(user_question)
print(answer)
end = time.time()
print('-'*30 + '\n')
print(f'Took {(end-start)/60} minutes')

The symptoms of bladder cancer can include being unable to urinate, lower back pain on one side, loss of appetite and weight loss, feeling tired or weak, swelling in the feet, and bone pain, as well as blood in the urine or other urinary symptoms that cause a person to see a health care provider. Additionally, a digital rectal exam (DRE) or pelvic exam may be performed during a physical exam to feel for any tumors or abnormalities in the bladder or pelvic area.
------------------------------

Took 0.3438388228416443 minutes


Question, present in the knowledge base

In [592]:
user_question = "My friend got diagnosed with adrenal cancer. What is the treatment?"

import time
import re

start = time.time()
answer = await answer_question(user_question)
print(answer)
end = time.time()
print('-'*30 + '\n')
print(f'Took {(end-start)/60} minutes')

The main treatment for adrenal cancer is an adrenalectomy, which involves removing the adrenal gland and any affected tissues or organs. The surgery can be performed through an incision in the back or the front of the abdomen, depending on the size and location of the tumor. Chemotherapy (chemo) may also be used in combination with an adrenalectomy, particularly if the cancer has spread to other parts of the body. Mitotane, a chemotherapy drug, is often used to treat adrenal cancer that has spread beyond the adrenal gland.
------------------------------

Took 0.7370374997456869 minutes


Question not present in the knowledge base

In [593]:
user_question = "How to treat skin cancer?"

import time
import re

start = time.time()
answer = await answer_question(user_question)
print(answer)
end = time.time()
print('-'*30 + '\n')
print(f'Took {(end-start)/60} minutes')

Sorry, I don't know the answer to "How to treat skin cancer?"
------------------------------

Took 0.008040090401967367 minutes


Question, present in the knowledge base

In [596]:
user_question = "What is anal cancer?"

import time
import re

start = time.time()
answer = await answer_question(user_question)
print(answer)
end = time.time()
print('-'*30 + '\n')
print(f'Took {(end-start)/60} minutes')

Anal cancer screening may be recommended for individuals with high-risk factors such as a history of HPV infection, a weakened immune system, or a family history of anal cancer. Screening may include a digital rectal exam (DRE) and/or an anal Pap test to check for abnormal cell changes in the anus. Early detection through screening can improve the chances of successful treatment and survival for individuals with anal cancer. Additionally, if anal cancer is suspected, exams and tests will be needed to confirm the diagnosis. If cancer is found, more tests will be done to help determine the extent (stage) of the cancer. A thorough medical history and physical exam can help identify risk factors and symptoms that may indicate the presence of anal cancer. In some cases, anal cancer may extend from one area into the other, making it difficult to determine the exact location of the cancer. Squamous cell carcinoma is the most common type of anal cancer, and it starts in the squamous cells that